# Storing Telemetry Data into DynamoDB


Recover data from the previous steps, make sure you've been through them and understand what they are:

In [1]:
#Hour and minute when this started to create unique names
%store -r unique
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A sample message payload
%store -r payload

Set the name for the blood pressure table:

**Extra Credit** If you have already done the [Alexa Nursing Skill Workshop](https://github.com/InternetOfHealthcare/nursing-alexa-skill-workshop) feel free to use the same table by adapting the table name below and skip table creation.

In [3]:
table_name = "bp_table_{}".format(unique)
table_name

'bp_table_ehw083110'

Create a DynamoDB table to store our messages. 
Notice that DynamoDB does not require you to provision or manage servers.
It will provision the infrastructure required to maintain the provisioned throughput (ReadCapacityUnits/WriteCapacityUnits).

In [4]:
!echo Creating table "{table_name}" this takes a few seconds...
table_arn = !aws dynamodb create-table \
  --table-name {table_name} \
  --attribute-definitions \
    AttributeName=name,AttributeType=S \
    AttributeName=time_stamp,AttributeType=N \
  --key-schema AttributeName=name,KeyType=HASH AttributeName=time_stamp,KeyType=RANGE \
  --provisioned-throughput ReadCapacityUnits=5,WriteCapacityUnits=5 \
  --query TableDescription.TableArn \
  --output text
table_arn = table_arn.s
!aws dynamodb wait table-exists --table-name {table_name}
!echo Table created "{table_arn}"

Creating table bp_table_ehw083110 this takes a few seconds...
Table created arn:aws:dynamodb:us-east-1:030555009967:table/bp_table_ehw083110


Create a rule capturing messages from the topic. The rule conditions are expressed in SQL statements like the one below. Rule statements works like this:


**SELECT** clause defines what fields to select from or add to the message.

**FROM** clause defines which topics to listen.

**WHERE** clause defines under what conditions messages should trigger this rule.


Note that they can not only capture but also enrich data and execute functions, see the [AWS IoT SQL Reference
](https://docs.aws.amazon.com/iot/latest/developerguide/iot-sql-reference.html) for more information.

In [5]:
sql = """
SELECT name, 
    time_stamp, 
    systolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{}'
WHERE 1=1
""".format(topic_name)
print(sql)


SELECT name, 
    time_stamp, 
    systolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM 'bp_topic/data'
WHERE 1=1



The rule also requires an IAM Role authorizing it to write data on DynamoDB. Let's create an IAM role with a trust policy specifying that this role can be assumed by AWS IoT:

In [6]:
# %load iot-trust.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "iot.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'iot.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

In [7]:
role_name = "bp2dynamodb_role_{}".format(unique) 
rule_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://iot-trust.json \
    --query Role.Arn \
    --output text
rule_role_arn = rule_role_arn.s
rule_role_arn

'arn:aws:iam::030555009967:role/bp2dynamodb_role_ehw083110'

The role is created but not authorized to do anything yet.
This time we'll be more restrictive in the security policy, specifying that the AWS IoT Rule assuming this IAM Role will be allowed to invoke ```dynamodb:PutItem``` only for the blood pressure table.

In [10]:
import json
rule_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Action": "dynamodb:PutItem",
        "Resource": table_arn
    }]
}
rule_policy = json.dumps(rule_policy, indent = 4)
print(rule_policy)

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "dynamodb:PutItem",
            "Resource": "arn:aws:dynamodb:us-east-1:030555009967:table/bp_table_ehw083110"
        }
    ]
}


In [11]:
policy_name="bp2dynamodb_policy_{}".format(unique)
policy_arn = !aws iam create-policy \
  --query Policy.Arn \
  --output text \
  --policy-name '{policy_name}' \
  --policy-document '{rule_policy}'
policy_arn = policy_arn.s
policy_arn

'arn:aws:iam::030555009967:policy/bp2dynamodb_rule_policyehw083110'

In [12]:
!aws iam attach-role-policy \
    --role-name {role_name} \
    --policy-arn {policy_arn}

AWS IoT Rules are created by a rule payload file binding the SQL satement to an action and role:

In [16]:
rule = {
    "ruleDisabled": False,
    "sql": sql,
    "description": "Send blood preasure measurements to DynamoDB",
    "actions": [{
        "dynamoDBv2": {
            "roleArn": rule_role_arn,
            "putItem": {
                "tableName": table_name
            }
        }
    }]
}
rule = json.dumps(rule, indent = 4)
rule_file="{}/bp2dynamodb_rule.json".format(home)
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

'/home/ec2-user/bp2dynamodb_rule.json'

In [17]:
print(open(rule_file).read())

{
    "ruleDisabled": false,
    "sql": "\nSELECT name, \n    time_stamp, \n    systolic, \n    diastolic, \n    id, \n    timestamp() as recv_stamp \nFROM 'bp_topic/data'\nWHERE 1=1\n",
    "description": "Send blood preasure measurements to DynamoDB",
    "actions": [
        {
            "dynamoDBv2": {
                "roleArn": "arn:aws:iam::030555009967:role/bp2dynamodb_role_ehw083110",
                "putItem": {
                    "tableName": "bp_table_ehw083110"
                }
            }
        }
    ]
}


In [18]:
rule_name="bprule{}".format(unique)
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}

# Triggering the topic rule

In [20]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(unique)
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(600)
mqtt.connect()
mqtt

In [21]:
data = {"id": "fa91d2de-24a1-4052-8b58-4f2ab743f20a",
        "name": "Martha Jones", 
        "systolic": 110, 
        "diastolic": 70, 
        "time_stamp": 1541059348245}
import json
msg = json.dumps(data, indent=4)
mqtt.publish(topic_name, msg , QoS = 1)

True

Lets check how many records have arrived on DynamoDB...

In [22]:
#Wait for it
import time
time.sleep(1)

In [23]:
!aws dynamodb scan --table-name {table_name} --query Count

1


In [24]:
mqtt.disconnect()

True

Visit the DynamoDB Console to visualize table data:

In [25]:
print("https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected={};tab=items".format(table_name))

https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:selected=bp_table_ehw083110;tab=items


All right, data is persisted now.

But what if someone has high blood pressure, better let them know, right?

Let's do that in the [Reacting to Events notebook](aws-iot-react-events.ipynb)